In [1]:
library(dplyr)
library(stringr)
library(readr)
library(tidyr)
library(lfe)
fp <- '/nfs/sloanlab004/projects/covid_mobility_proj/data/'
sci <- read_delim(str_c(fp, 
                        'fb_social_connectedness/sci_county/county_county_data.tsv'), delim ='\t', 
                  col_types = 'ddd')
safegraph <- read_csv(str_c(fp, 'PROCESSED_DATA/safegraph_social_distancing_aggregate_county.csv'))
counties_long <- read_csv(str_c(fp, 'PROCESSED_DATA/county_policy_long.csv'))
census_data <- read_csv(str_c(fp, 'census_data/cc-est2018-alldata.csv'))


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Parsed with column specification:
cols(
  county_fips = col_double(),
  device_count = col_double(),
  completely_home_device_count = col_double(),
  part_time_work_behavior_devices = col_double(),
  full_time_work_behavior_devices = col_double(),
  ds = col_date(format = "")
)

Parsed with column specification:
cols(
  fips = col_double(),
  name = col_character(),
  state_abbv = col_character(),
  type = col_character(),
  dt = col_date(format = "")
)

Parsed with column specification:
cols(
  .default = col_double(),
  SUMLEV = col_character(),
  STATE = col_character(),
  COUNTY = col_character(),
  STNAME = col_character(),
  CTYNAME = co

In [2]:
counties_long %>% 
    mutate(value = 1) %>%
    spread(key = type, value = value) -> countyPolicy

safegraph %>%
    group_by(county_fips) %>%
    summarize(n = n()) %>%
    filter(n == max(n)) %>%
    select(county_fips) %>%
    inner_join(sci %>% ungroup() %>% select(county_fips = user_county) %>% distinct()) %>%
    inner_join(countyPolicy %>% select(county_fips = fips) %>% distinct()) -> fips

census_data %>% 
  filter(YEAR == 11) %>%
  group_by(STATE, COUNTY) %>% 
  summarise(n = sum(TOT_POP)) %>% 
  ungroup() %>%
  mutate(COUNTY = str_c(STATE, COUNTY)) %>%
  rename(county_fips = COUNTY) %>% 
  select(-STATE) %>% 
  mutate(county_fips = as.numeric(county_fips)) -> population

Joining, by = "county_fips"

Joining, by = "county_fips"



In [3]:
sci %>%
    inner_join(fips, by = c('user_county' = 'county_fips')) %>%
    inner_join(fips, by = c('fr_county' = 'county_fips')) %>%
    mutate(user_state_fips = floor(user_county/1000), 
           fr_state_fips = floor(fr_county/1000), 
           sameState_sci = ifelse(user_county == fr_county, 0 ,
                                ifelse(user_state_fips == fr_state_fips, scaled_sci, 0)),
           outState_sci = ifelse(user_state_fips == fr_state_fips, 0, scaled_sci)) %>%
    group_by(user_county) %>%
    mutate(sameState_w = sameState_sci/sum(sameState_sci),
           outState_w = outState_sci/sum(outState_sci)) %>% 
    arrange(user_county, fr_county) -> sci

In [4]:
sci %>%
    inner_join(fips, by = c('user_county' = 'county_fips')) %>%
    inner_join(fips, by = c('fr_county' = 'county_fips')) %>%
    left_join(population, by = c('fr_county' = 'county_fips')) %>%
    mutate(w = ifelse(user_county == fr_county, 0 , scaled_sci * n)) %>%
    group_by(user_county) %>%
    mutate(w = w/sum(w)) %>% 
    arrange(user_county, fr_county) -> sci_weighted

In [17]:
sci_weighted %>% 
  left_join(population, by = c('user_county' = 'county_fips')) %>% 
  mutate(user_state = floor(user_county/1000),
         fr_state = floor(fr_county/1000)) %>% 
  group_by(user_county, user_state, fr_state, n.y) %>% 
  summarise(w = sum(w)) %>% 
  ungroup() %>% 
  group_by(user_state, fr_state) %>% 
  mutate(total_n = sum(n.y)) %>% 
  summarise(w = sum(w*n.y/total_n)) %>% 
  ungroup() %>% 
  mutate(w = ifelse(user_state == fr_state, 0, w)) %>% 
  group_by(user_state) %>% 
  mutate(w = w/sum(w)) %>% 
  ungroup() %>% 
  spread(key = fr_state, value= w) %>% 
  ungroup() %>% 
  select(-user_state) %>% 
  as.matrix() -> social_adj_matrix_state

In [20]:
sci_weighted %>% 
    select(user_county, fr_county, w) %>%
    spread(key = fr_county, value = w) %>%
    ungroup() %>%
    select(-user_county) %>%
    as.matrix() -> social_adj_matrix

In [21]:
save(social_adj_matrix, social_adj_matrix_state,
     file='/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/social_adj_matrix.RData')